# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ballina,-28.8667,153.5667,18.68,82,0,7.46,AU,1697125066
1,1,ilhabela,-23.7781,-45.3581,26.57,77,64,2.18,BR,1697125066
2,2,blackmans bay,-43.0167,147.3167,10.07,81,99,2.34,AU,1697125066
3,3,shimoda,34.6667,138.9500,16.72,75,11,4.07,JP,1697125066
4,4,puerto natales,-51.7236,-72.4875,6.91,86,75,4.33,CL,1697125066


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
                                        "Lng",
                                        "Lat",
                                        geo = True,
                                        color = "City",
                                        alpha = 0.9,
                                        size = "Humidity",
                                        tiles = "OSM"
                                    ) 
       
   

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_weather_df = city_data_df

#A max temperature lower than 27 degrees but higher than 21
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 27) & (filtered_weather_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Wind Speed"] < 4.5)]
#Zero cloudiness
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

# Display sample data
filtered_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,ain sukhna,29.6002,32.3167,26.63,43,0,2.32,EG,1697125066
29,29,nanakuli,21.3906,-158.1547,24.50,77,0,2.57,US,1697125069
64,64,jarabulus,36.8175,38.0111,26.19,28,0,1.85,SY,1697125073
209,209,pozzallo,36.7305,14.8499,26.36,65,0,2.72,IT,1697125090
217,217,turpan,42.9333,89.1667,23.08,20,0,0.89,CN,1697125091
303,303,saint-francois,46.4154,3.9054,26.69,39,0,1.54,FR,1697125102
315,315,stornara,41.2873,15.7698,26.82,45,0,4.04,IT,1697125104
408,408,almansa,38.8692,-1.0971,25.42,29,0,2.68,ES,1697125115
423,423,al bawiti,28.3492,28.8659,25.54,35,0,4.16,EG,1697125117
461,461,haiku-pauwela,20.9219,-156.3051,23.91,78,0,2.57,US,1697125122


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,ain sukhna,EG,29.6002,32.3167,43,
29,nanakuli,US,21.3906,-158.1547,77,
64,jarabulus,SY,36.8175,38.0111,28,
209,pozzallo,IT,36.7305,14.8499,65,
217,turpan,CN,42.9333,89.1667,20,
303,saint-francois,FR,46.4154,3.9054,39,
315,stornara,IT,41.2873,15.7698,45,
408,almansa,ES,38.8692,-1.0971,29,
423,al bawiti,EG,28.3492,28.8659,35,
461,haiku-pauwela,US,20.9219,-156.3051,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key,
    }


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] =  f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ain sukhna - nearest hotel: العين السخنة
nanakuli - nearest hotel: Aulani, A Disney Resort & Spa
jarabulus - nearest hotel: No hotel found
pozzallo - nearest hotel: Hotel Pozzallo
turpan - nearest hotel: Oriental
saint-francois - nearest hotel: Chez Lily
stornara - nearest hotel: Herdonia Hotel
almansa - nearest hotel: Casa Almántica
al bawiti - nearest hotel: Old Oasis Hotel
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
kozluk - nearest hotel: Kozluk Öğretmenevi
houyu - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
5,ain sukhna,EG,29.6002,32.3167,43,العين السخنة
29,nanakuli,US,21.3906,-158.1547,77,"Aulani, A Disney Resort & Spa"
64,jarabulus,SY,36.8175,38.0111,28,No hotel found
209,pozzallo,IT,36.7305,14.8499,65,Hotel Pozzallo
217,turpan,CN,42.9333,89.1667,20,Oriental
303,saint-francois,FR,46.4154,3.9054,39,Chez Lily
315,stornara,IT,41.2873,15.7698,45,Herdonia Hotel
408,almansa,ES,38.8692,-1.0971,29,Casa Almántica
423,al bawiti,EG,28.3492,28.8659,35,Old Oasis Hotel
461,haiku-pauwela,US,20.9219,-156.3051,78,Inn At Mama's Fish House


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   frame_width=500,
                                   frame_height=500,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)